# Black Magic AI Detectron2 Cloud Vision API Tutorial - lambda function

<img src="../images/blackmagicailogo.png">

This tutorial demonstrats how to create the lambda zip package code that is uploaded to create the AWS Lambda function that is triggered by the AWS API Gateway. This Lambda in turn invokes the Detectron2 model deployed to an AWS Sagemaker endpoint.

You can make a copy of this tutorial by "File -> Open in playground mode" and make changes there. __DO NOT__ request access to this tutorial.

# Create AWS Lambda Deployment package

Create aws Lamda deployment zip package
#Ref:

https://docs.aws.amazon.com/lambda/latest/dg/lambda-deploy-functions.html

https://docs.aws.amazon.com/lambda/latest/dg/python-package.html#python-package-create-package-no-dependency

Lambda function code

In [ ]:
%%writefile "~/SageMaker/lamdaenv/lambda_function.py"
import os
import json
import boto3
from base64 import b64encode,b64decode
from io     import BytesIO
from PIL import Image

# grab environment variables
ENDPOINT_NAME = os.environ['ENDPOINT_NAME']

# Ref:
# https://docs.aws.amazon.com/lambda/latest/dg/lambda-deploy-functions.html
# https://docs.aws.amazon.com/lambda/latest/dg/python-package.html#python-package-create-package-no-dependency
def lambda_handler(event, context):
    runtime= boto3.client('runtime.sagemaker')
    # Base64 decode data it came in encoded
    img_string = event.get("body","")
    img_data = b64decode(img_string)

    img_buffer = BytesIO(img_data)
    img_buffer.seek(0)
    image_src = Image.open(img_buffer)
    
    imgByteArr = BytesIO()
    image_src.save(imgByteArr, format=image_src.format)
    imgByteArr = imgByteArr.getvalue()
        
    # Send image via InvokeEndpoint API
    response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME, ContentType='application/x-image', Body=imgByteArr)
    result = response['Body'].read().decode()
    return result

In [ ]:
# lambda template
# Create function from scratch then create function from zip file
# Add ENDPOINT_NAME configuaration variable to lambda after creating it
# use python 3.6 & Role panoptic-role-7ixf6wlv
%cd ~
%cd SageMaker/lamdaenv

# !pip show Pillow | grep Location:
%cd ~/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages
!zip -r ../../../../../../SageMaker/lamdaenv/lambda-deployment-package.zip PIL Pillow-8.4.0.dist-info Pillow.libs
%cd ../../../../../../SageMaker/lamdaenv
!zip -g lambda-deployment-package.zip lambda_function.py